![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

# 1. Load the dataset
df = pd.read_csv("rental_info.csv")

# 2. Create rental_length_days column
df["rental_date"] = pd.to_datetime(df["rental_date"])
df["return_date"] = pd.to_datetime(df["return_date"])
df["rental_length_days"] = (df["return_date"] - df["rental_date"]).dt.days

# 3. Create dummy variables for special_features
#df["deleted_scenes"] = (df["special_features"] == "Deleted Scenes").astype(int)
#df["behind_the_scenes"] = (df["special_features"] == "Behind the Scenes").astype(int)

# correctly create dummy flags (handles NaNs and multi-feature strings)
df["deleted_scenes"] = df["special_features"].fillna("").str.contains("Deleted Scenes", regex=False).astype(int)
df["behind_the_scenes"] = df["special_features"].fillna("").str.contains("Behind the Scenes", regex=False).astype(int)

# quick sanity check
print("deleted_scenes ones:", df["deleted_scenes"].sum())
print("behind_the_scenes ones:", df["behind_the_scenes"].sum())


# 4. Define features (avoid data leakage — don't use return_date or rental_length_days)
features_to_exclude = ["rental_date", "return_date", "special_features", "rental_length_days"]
X = df.drop(columns=features_to_exclude)
y = df["rental_length_days"]

# 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=9
)

# 6. Try multiple regression models
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=9),
    "Lasso": Lasso(alpha=0.01, random_state=9),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=9)
}

best_model = None
best_mse = float("inf")

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name}: MSE = {mse:.4f}")
    
    if mse < best_mse:
        best_mse = mse
        best_model = model

# 7. Output the best model and its MSE
print("\nBest Model:", best_model.__class__.__name__)
print("Best MSE:", best_mse)


deleted_scenes ones: 7888
behind_the_scenes ones: 8507
LinearRegression: MSE = 2.9417
Ridge: MSE = 2.9418
Lasso: MSE = 2.9503
RandomForest: MSE = 2.0301

Best Model: RandomForestRegressor
Best MSE: 2.030141907417274
